In [11]:
import numpy as np
import pandas as pd
import scipy.sparse as ss
import pickle
import scipy.io as sio

In [12]:
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',1000)
pd.set_option('display.max_colwidth',1000)
#pd.set_option('display.max_rows',30)
#
df_train=pd.read_csv('../data1/train.csv')
print('用户：',df_train['msno'].unique().shape[0])
print('歌曲：',df_train['song_id'].nunique())

用户： 30755
歌曲： 359966


In [13]:
#数据预处理
#type={'msno','song_id','source_system_tab','source_screen_name','source_type','target'}
#去掉'source_system_tab','source_screen_name','source_type'三种不相关的特征
df_train=df_train[['msno','song_id','target']]
print('去掉无用的特征后训练集维度：',df_train.shape)

去掉无用的特征后训练集维度： (7377418, 3)


In [14]:
#特征工程
#msno song_id属于hash编码后的特征
#计算用户对每首歌曲的打分
df_user_rating=df_train[['msno','target']].groupby('msno').sum().reset_index()

df_user_rating.rename(columns={'target':'total_rating'},inplace=True)
print(df_user_rating,)

                                               msno  total_rating
0      ++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=           293
1      ++AH7m/EQ4iKe6wSlfO/xXAJx50p+fCeTyF90GoE9Pg=           141
2      ++e+jsxuQ8UEnmW40od9Rq3rW7+wAum4wooXyZTKJpk=            76
3      ++xWL5Pbi2CpG4uUugigQahauM0J/sBIRloTNPBybIU=             1
4      +/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=            82
5      +/USTSczbe/vrPPatLFT8TPKNBt0J5ValJhfr3vNALI=             0
6      +/UwoUi5+rNj/F6RO6gMrMhOy0oTzs90MWKVNZs4+Wg=            96
7      +/tv7+daiSsEEBhNDnQZL32wdfmr2GSYmX2aQPGONRY=             3
8      +/vppV5BoMKMNAk7Bej5rjhKeE6VqrRjUv8EYXoAjyM=             2
9      +0+XaewkS3za58vHwKCVGwRYRvjig4lTtKDV/tOkzbU=            49
10     +06pc1V8QfywrGOTNln/7X7Md8dA8fVOIU37n2M5mv8=           349
11     +0B4aHABar5ltaWDG4M6KCzNxpD6wnsvZpvh+o9KgtE=           116
12     +0CxUZKWtmqPKlW+2VF+hY2rfq+Dh9JM/AqoZJnCr4E=            21
13     +0U8n45rrg6b5WCy3wpqOUYCwZyLqecr3lux9K6ahMk=           247
14     +0e

In [15]:
#每首歌曲的播放比例
df_train=pd.merge(df_train,df_user_rating)
del df_user_rating
print('用户订阅过的音乐，及总和：\n',df_train)

用户订阅过的音乐，及总和：
                                                  msno                                       song_id  target  total_rating
0        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=       1          2791
1        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=       1          2791
2        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=       1          2791
3        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  bPIvRTzfHxH5LgHrStll+tYwSQNVV8PySgA3M1PfTgc=       1          2791
4        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  XUz4Z4wPPChz+OIwkwj7HJ8teIjW3rEMdEATM80rDxM=       1          2791
5        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  +h+uIXLQyXHuCdwAY5oT3T302vt0IdQJtux0APXhriw=       1          2791
6        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  +LztcJcPEEwsikk6+K5udm06XJQMzR4+lzavKLUyE0k=       1          2791
7        

In [16]:
#删除总打分次数为0的用户（这里打分此时为0，代表着该用户在本月是第一次来
# 或者该用户上个月订阅过音乐，但是这个月该用户流失了
#通过观察，发现索引为7377417的用户订阅的音乐次数为0，所以去掉该用户
#total_rating为0的索引
index=df_train[df_train.total_rating==0].index.tolist()
#print('index=',len(index))
df_train=df_train.drop(index=index)

In [18]:
print(df_train.sort_values(by=['total_rating'],ascending=False))
df_train['fractional_rating_count']=df_train['target']/df_train['total_rating']
print(df_train)

                                                 msno                                       song_id  target  total_rating  fractional_rating_count
2907879  o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=  LPJOSGYnGUFOXJkKPrmEXJ7iMb3mvY8R2ofkmgU71LA=       1          3288                 0.000304
2904831  o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=  TNX7iJ97he2nSjHX/EqW5BYXIYoQjwjf2Hg/CHYqR80=       1          3288                 0.000304
2904824  o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=  rOWFaCuk3hjksL6KjP7fNvquyV/kbP7z5yS+pCm20KA=       0          3288                 0.000000
2904825  o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=  hKs8gwK7qKNeXTvzZ3U8y8aDiBsE0y7uksji0TPbnrQ=       1          3288                 0.000304
2904826  o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=  m/xwFbdL6o4DK0xAo+PgYVGptZIJd6zlq+IsX1LLg8U=       0          3288                 0.000000
2904827  o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=  V5vnSrmStEWpy2wuA/VqDbaSR99xnf6sQQapvSgysb4=       1          3

                                                 msno                                       song_id  target  total_rating  fractional_rating_count
0        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=       1          2791                 0.000358
1        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=       1          2791                 0.000358
2        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=       1          2791                 0.000358
3        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  bPIvRTzfHxH5LgHrStll+tYwSQNVV8PySgA3M1PfTgc=       1          2791                 0.000358
4        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  XUz4Z4wPPChz+OIwkwj7HJ8teIjW3rEMdEATM80rDxM=       1          2791                 0.000358
5        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  +h+uIXLQyXHuCdwAY5oT3T302vt0IdQJtux0APXhriw=       1          2

In [19]:
#所有的用户和item
users=df_train['msno'].unique()
n_users=len(users)
print('用户数量为：',n_users)
items=df_train['song_id'].unique()
n_items=len(items)
print('歌曲数量为：',n_items)

用户数量为： 27113
歌曲数量为： 359353


In [20]:
#存为csv格式
feat_names=df_train.columns
train1 = pd.DataFrame(columns = feat_names, data = df_train)
train1.to_csv('train1.csv',index = False,header=True)

In [21]:
print(train1)

                                                 msno                                       song_id  target  total_rating  fractional_rating_count
0        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=       1          2791                 0.000358
1        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=       1          2791                 0.000358
2        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=       1          2791                 0.000358
3        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  bPIvRTzfHxH5LgHrStll+tYwSQNVV8PySgA3M1PfTgc=       1          2791                 0.000358
4        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  XUz4Z4wPPChz+OIwkwj7HJ8teIjW3rEMdEATM80rDxM=       1          2791                 0.000358
5        FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=  +h+uIXLQyXHuCdwAY5oT3T302vt0IdQJtux0APXhriw=       1          2